In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math as mt
import seaborn as sns
from tqdm import tqdm
import pandas as pd
from hangul_utils import split_syllables, join_jamos
from tqdm.auto import tqdm
import re
#from eunjeon import Mecab
from konlpy.tag import Mecab
from hanspell import spell_checker

In [2]:
con_dict = [
    
    ['ㅏㅣ','ㅐ'], ['ㅑㅣ','ㅒ'], ['ㅓㅣ','ㅔ'],
    ['ㅕㅣ','ㅖ'], ['ㅗㅣ','ㅚ'], ['ㅗㅐ','ㅙ'],
    ['ㅜㅓ','ㅝ'], ['ㅜㅔ','ㅞ'], ['ㅡㅣ','ㅢ'],
    ['ㅣㅏ','ㅑ'], ['ㅣㅓ','ㅕ'], ['ㅣㅗ','ㅛ'],
    ['ㅣㅜ','ㅠ'], ['ㅡㅓ','ㅓ'], ['ㅗㅏ','ㅘ']
    
]

jongsung_list = [ 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']


#lis_beta = ['EP+EF', 'VCP+EF', 'B+EF', 'B+EP+EF', 'B+VCP+EF', 'EF','EP']

lis_beta = ['EP+EF', 'EF', 'B+EF', 'B+EP+EF']

#어말을 처리해 주기 위한 것으로, 나중에 EC등이 필요해 진다면 이 부분에 EC 등을 집어넣어준다. 참고로 말하자면 이는 마지막에 위치해야한다.
#특이 바로 밑의 이 부분은 형태소 태그가 이 리스트 안의 것과 일치하는 경우 단순 삭제를 하는 것이고
lis_beta_ef = ['EP+EP+EF', 'EP+EF', 'EF', 'UNKNOWN']

lis_beta_ef_h = ['EF', 'UNKNOWN']

#이 부분 같은 경우는 마지막에 오는 것을 처리하는 것으로 단독으로만 들어가는게 좋겠지?
lis_tag_last = ['EF', 'UNKNOWN']

lis_end = [
    
    'ㅅㅡㅂㄴㅣㄷㅏ',
    'ㅂㄴㅣㄷㅏ', 'ㄴㅣㄷㅏ', 'ㅂㅅㅣㄷㅏ', 'ㅅㅣㄷㅏ', 'ㄹㄹㅐㅇㅛ',
    'ㅅㅔㅇㅛ', 'ㄷㅔㅇㅛ', 'ㅇㅔㅇㅛ', 'ㅇㅖㅇㅛ', 'ㄴㅏㅇㅛ', 'ㄱㅜㄴㅇㅛ','ㅇㅡㄹㄲㅏㅇㅛ', 'ㅇㅣㄹㄲㅏㅇㅛ', 'ㄹㄲㅏㅇㅛ', 'ㅇㅡㄴㄱㅏㅇㅛ', 'ㅇㅣㄴㄱㅏㅇㅛ',
    'ㄱㅗㅇㅛ', 'ㅇㅛ',
    'ㅈㅛ',
    'ㅅㅣㅂㅅㅣㅇㅗ', 'ㅅㅣㅇㅗ', 'ㅇㅗ',
    'ㅂㄴㅣㄲㅏ',  
    
]

lis_end_2low = [
    
    'ㄷㅓㄹㅏ','ㄴㅡㄴㄷㅏ','ㄴㄷㅏ', 'ㅆㄷㅏ', 'ㄹㅗㄷㅏ', 'ㄷㅏ', 'ㄱㅔ', 'ㅈㅣ', 'ㅈㅏ', 'ㄴㅑ', 
    'ㄹㅏ',
    'ㅇㅑ', 
    'ㄴㅣㄲㅏ', 
    'ㄲㅏ', 
    'ㄴㅣ', 
    'ㅇㅏ', 
    'ㄷㅔ', 
    'ㅇㅓ', 
    'ㄴㅏ', 
    
]

lis_ic = ['ㅇㅖ', 'ㄴㅔ', 'ㅇㅏㄴㅣㅇㅗ', 'ㅇㅏㄴㅣㅇㅛ']

lis_wk = [
    
    ['ㄱㅖ', 'ㅇㅣㅆㅇㅡ'], ['ㅈㅜㅁㅜ','ㅈㅏ'], ['ㅈㅏㅂㅅㅜ','ㅁㅓㄱㅇㅡ']
    
]

# lis_end_2low = [
    
#     'ㄷㅓㄹㅏ','ㄴㄷㅏ', 'ㅆㄷㅏ', 'ㄷㅏ', 
#     'ㄹㅏ',
#     'ㅇㅑ', 
    
# ]

P_LIST = ['.', '?', '!', '\'', '\"', 'ᆞ', 'ᆢ', 'ㆍ',  '”', '’',')', '(', ',', '”']

SV_LIST = ['\'', '\"', ':', ';', '?']

lis_plus = [
    
    'EP', 'VCP', 
    
]

In [3]:
mec = Mecab()

In [4]:
def detect_h(input, lis_end_h, lis_end_l):
    for i in lis_end_h:
        if len(input)>=len(i):
            if input[-len(i):]==i:
                return 1
            
    for i in lis_end_l:
        if len(input)>=len(i):
            if input[-len(i):]==i:
                return 0
            
    return -1

def unite(input, dict):
    for i in dict:
        input = re.sub(i[0],i[1],input)
    return input
    
## 자모 단위로 문장을 나누고 합칠 때 쓰는 class ##
class Jamodealer:
    jamo = []
    pp = ''
    #각 단어들을 받아와서 자모단위로 나눈다.
    def __init__(self,lis_word):
    
        self.jamo = []
        for i in lis_word:
            self.jamo.append(split_syllables(i))
    
    ##사전에서 변환된 자모단위로 분리된 문장을 합칠 때 쓰는 함수이다.     
    def make_one(self):
        #list 형태로 저장된 자모들의 집합을 하나의 string pp에 저장한다. 
        self.pp = ''
        for i in self.jamo:
             self.pp= self.pp+i
        ##종성과 종성을 합쳐야 하는 경우가 있다면 합친다.        
        self.pp = unite(self.pp, con_dict)
        
        #자모 단위의 string에서 자모 단위로 사전을 만들고 거기에 index를 부여한다.        
        chars = list(set(self.pp))
        char_to_ix = { ch:i for i,ch in enumerate(chars) }
        ix_to_char = { i:ch for i,ch in enumerate(chars) }
        
        #자모 단위로 분리되었던 문장을 다시 하나로 합친다.
        jamo_numbers = [char_to_ix[x] for x in self.pp]
        restored_jamo = ''.join([ix_to_char[x] for x in jamo_numbers])
        #합쳐진 문장을 return 한다.
        restored_text = join_jamos(restored_jamo)
        return restored_text

In [5]:
def prepro_ch01(input, lis_ef, tag_last, lis_w_last, lis_w_last_not):
    
    lis_res_word = []
    
    lis_input = input.split()
    lis_word = []
    lis_tag = []
    lis_last_word = []
    lis_ind = []
    lis_ind_ele = []
    
    lis_result = []
    
    for i in range(len(lis_input)):
        ele_w = []
        ele_t = []
        
        an = mec.pos(lis_input[i])
        for j in range(len(an)):
            ele_w.append(an[j][0])
            ele_t.append(an[j][1])
        elem_w = ' '.join(ele_w)
        elem_t = '/'.join(ele_t)
        
        jam_pre = Jamodealer(elem_w)
        lis_word.append(''.join(jam_pre.jamo))
        lis_tag.append(elem_t)
    
    for i in range(len(lis_tag)):
        
        if 'EF/SF' in lis_tag[i] or 'EF/SV' in lis_tag[i] or 'EC/SY'  in lis_tag[i] or 'EC/SF' in lis_tag[i] or 'EC/SSO' in lis_tag[i]:

            elemen_t = lis_tag[i].split('/')
            elemen_w = lis_word[i].split(' ')
            flag = 0
            ff=0
            
#             print('iiii')
            
            for j in range(len(elemen_t)):

                flag_end = detect_h(elemen_w[j], lis_w_last_not, lis_w_last)
                

                
                if (flag_end == 0 and 'EF' in elemen_t[j]):
#                     print('eeee')
                    for jam in lis_w_last:
                        
                        if len(elemen_w[j])>=len(jam):
                            
                            if elemen_w[j][-len(jam):]==jam:
                                
                                
                                lis_ind.append(i)
                                lis_last_word.append(jam)
                                elemen_w[j] = elemen_w[j][:-len(jam)]
                                elemen_w[j] = elemen_w[j] + '__+__'
                                ff=1
                                break
                                
                elif 'VV+EF' in elemen_t[j] or 'VA+EF' in elemen_t[j] or 'XSV+EF' in elemen_t[j] or 'VCP+EF' in elemen_t[j]:
#                     print('qq')
                    if flag_end == -1:
                        lis_ind.append(i)
                        lis_last_word.append('')
                        elemen_w[j] = elemen_w[j] + '__+__'
                    
                    
            res_w = ''.join(elemen_w)
            lis_result.append(res_w)
                                
        else:
            
            rere = lis_word[i].split(' ')
            
            resres = ''.join(rere)
            
            lis_result.append(resres)
            
            
    return lis_result, lis_tag, lis_ind, lis_last_word
                

In [6]:
print(prepro_ch01('그것을 하고 만다.', lis_beta_ef, lis_tag_last, lis_end_2low, lis_end))

(['ㄱㅡㄱㅓㅅㅇㅡㄹ', 'ㅎㅏㄱㅗ', 'ㅁㅏㄴ__+__.'], ['NP/JKO', 'VV/EC', 'VA/EF/SF'], [2], ['ㄷㅏ'])


In [7]:
def convert_not(stc, tag, ex):
    ind_point = -1
    point = ''

#     for i in P_LIST:
#         if stc.find(i) !=-1:
#             point = i
#             break
            
    for i in range(len(stc)):
        if stc[i] in P_LIST:
            point = stc[i]
            break
            
    if point in P_LIST:
        ind_point = stc.index(point)
    
    r_word = ''
    r_pun = ''
    
    if ind_point!=-1:
        r_word = stc[:ind_point]
        r_pun = stc[ind_point:]
    else:
        r_word = stc
        
    return r_word + ex + r_pun

def convert_an(stc, tag, ex):
    
    ind_point = -1
    point = ''

#     for i in P_LIST:
#         if stc.find(i) !=-1:
#             point = i
#             break
            
#     for i in range(len(stc)):
#         if stc[len(stc) - i-1] in P_LIST:
#             point = stc[len(stc) - i-1]
#             break
    
#     for i in range(len(stc)):
#         if stc[i] in P_LIST:
#             ind_point = i
#             break
        
    ind_point = stc.index('__+__')
    stc = stc.replace('__+__', '', 1)
    
    r_word = ''
    r_pun = ''
    
    if ind_point!=-1:
        r_word = stc[:ind_point]
        r_pun = stc[ind_point:]

    else:
        r_word = stc
    
    if ex=='ㄹㅏ':
        
        if len(r_word)!=0:
            if 'ㅗ'==stc[-2]:
                return r_word+'ㅏㅇㅛ'+r_pun
            elif 'ㅎㅏ'==stc[-3:-1]:
                return r_word+'ㅣㅇㅛ'+r_pun
            elif 'ㄷㅏㄹ' in stc:
                return r_word+'ㄹㅏ'+r_pun
                if 'VA' not in tag:
                    return r_word+'ㄹㅏㅇㅛ'+r_pun
                return r_word[:-3]+'ㅈㅜㅅㅔㅇㅛ'+r_pun
            elif 'ㅗㄹ' == r_word[-2:]:
                return r_word+'ㄹㅏㅇㅛ'+r_pun
            else:

                return r_word+'ㅇㅛ'+r_pun
        else:
            return r_word+ex+r_pun

        #return r_word+'ㅇㅛ'+r_pun
    elif ex=='ㄱㅜㄴㅏ':
        return r_word+'ㄱㅜㄴㅇㅛ'+r_pun
    elif ex=='ㄴㅑ':
        return r_word+'ㄴㅏㅇㅛ'+r_pun
    elif ex=='ㅈㅏㄶㄴㅣ':
        return r_word+'ㅈㅏㄶㅇㅏㅇㅛ'+r_pun
    elif ex=='ㄱㅔ':###############################
        print(r_word)
        if 2<=len(r_word) and r_word[-2:]=='ㅎㅏ':
            return r_word+'ㅣㅇㅛ'+r_pun
        elif 2<=len(r_word) and r_word[-2:]=='ㅅㅣ':
            return r_word+'ㅓㅇㅛ'+r_pun
        return r_word+'ㄱㅔㅇㅛ'+r_pun
        
    elif ex=='ㅇㅓㄹㅏ':
        return r_word+'ㅇㅓㅇㅛ'+r_pun
    elif ex=='ㅇㅏㄹㅏ':
        return r_word+'ㅇㅏㅇㅛ'+r_pun
    elif ex=='ㄴㅗㄹㅏ':
        if stc[-3:-1]=='ㅎㅏ':
            return r_word+'ㅣㅇㅛ'+r_pun
        else:
            return r_word+'ㅇㅓㅇㅛ'+r_pun
    elif ex=='ㄷㅏㄷㅓㄹㅏ':
        return r_word+'ㄷㅐㅇㅛ'+r_pun
    elif ex=='ㄷㅓㄹㅏ':
        wd = ''
        if 'ㄷㅏ' in r_word:
#             print('dd')
            return r_word[:-1]+'ㅐㅇㅛ'+r_pun
        if r_word[-1] in jongsung_list:
            wd = r_word[-2]
        else:
            wd = r_word[-1]
        if wd=='ㅗ':
            return r_word+'ㅏㅇㅛ'+r_pun
        elif wd=='ㅜ':
            return r_word+'ㅓㅇㅛ'+r_pun
        elif wd=='ㅏ':
            return r_word+'ㅇㅏㅇㅛ'+r_pun
        else:
            return r_word+'ㅇㅓㅇㅛ'+r_pun
    elif ex=='ㄴㅣ' or ex =='ㄴㅡㄴㄱㅏ' or ex =='ㅇㅡㄴㄱㅏ':####################
        
        isVcp = tag[-11:-3]
        
        if isVcp =='VCP' or isVcp =='/VCP' or isVcp =='NNG/VCP':
            #print('www')
            return r_word + 'ㅇㅖㅇㅛ' + r_pun
        elif isVcp == 'VCP/NULL':
            return r_word + 'ㅇㅔㅇㅛ' + r_pun
        
        
        if ex=='ㄴㅣ' and stc[-3:-1]=='ㅌㅔ':
            return r_word+'ㄴㅣㅇㅛ'+r_pun
        
        if ex=='ㄴㅣ' and 'XSA/EF' in tag:
            return r_word+'ㄴㅏㅇㅛ'+r_pun
        
        if ex=='ㄴㅣ' and 'NNG' in tag:
            return r_word+'ㅇㅖㅇㅛ'+r_pun
        
        return r_word+'ㄴㅏㅇㅛ'+r_pun
    elif ex=='ㅇㅑ':
        return r_word+'ㅇㅔㅇㅛ'+r_pun
#     elif ex=='ㄷㅓㄹㅏ':
#         if 'ㄷㅏ' in stc:
#             return stc[:-1]+'ㅔㅇㅛ'
#         return stc+'ㄷㅔㅇㅛ'
    elif ex=='ㅈㅣ':
        return r_word+'ㅈㅛ'+r_pun
    elif ex=='ㅈㅏ':
        return r_word+'ㅈㅛ'+r_pun
    else:
#         print('dd')
        return r_word+ex+'ㅇㅛ'+r_pun
    
def convert_da(stc, tag, ex):
    #print(tag)
    if 'UNKNOWN/' in tag:
        isVcp = tag[-17:]
    else:
        isVcp = tag[-9:]

    isETM = tag[-12:]


    ind_point = -1
    point = ''

    ind_point = stc.index('__+__')
    stc = stc.replace('__+__', '', 1)
    
    r_word = ''
    r_pun = ''

    if ind_point!=-1:
        r_word = stc[:ind_point]
        r_pun = stc[ind_point:]
    else:
        r_word = ''
        r_pun = stc[0]
    
    try:
        jongsung = r_word[-1]
    
    except:
        return ex+r_pun
        
    if jongsung in jongsung_list:
        if  isETM.find('+ETM') != -1:
            r_word = r_word[:-1]
            final = r_word +'ㅂㄴㅣㄷㅏ'+r_pun
        else:
            final = r_word +'ㅅㅡㅂㄴㅣㄷㅏ'+r_pun
    else:
        if isVcp.find('VCP') != -1:
            if len(r_word)>=2:
                if 'ㅇㅣ' in r_word:
                    return r_word +'ㅂㄴㅣㄷㅏ'+r_pun
            final = r_word +'ㅇㅣㅂㄴㅣㄷㅏ'+r_pun
        elif 'SN/NULL' in tag:
            final = r_word +'ㅇㅣㅂㄴㅣㄷㅏ'+r_pun
        else:
            final = r_word +'ㅂㄴㅣㄷㅏ'+r_pun
    #print(final)
    return final

In [8]:
class Changer(object):
    
    def make_end_h(self, stc, tag, ex):

#         print(ex)
        if 'ㄷㅏ'== ex[-2:]:

            return convert_da(stc, tag, ex)
            #return convert_not(stc, tag, ex)
        else:
            #return convert_not(stc, tag, ex)
            return convert_an(stc, tag, ex)

    
    def to_high(self, input):
        
        if '  ' in input:
            return input
        
        result = input
        
        

        
        lis_w, lis_t, lis_target_ind, off_word = prepro_ch01(result, lis_beta_ef, lis_tag_last, lis_end_2low, lis_end)

        
        if len(lis_target_ind)!=0:

            for i in range(len(lis_target_ind)):

                new_end = self.make_end_h(lis_w[lis_target_ind[i]], lis_t[lis_target_ind[i]], off_word[i])
                lis_w[lis_target_ind[i]] = new_end
            
            
#             jam.jamo.append(w_last)
            
            res = ' '.join(lis_w)
            jam = Jamodealer(res)
            
            
            return jam.make_one()

        return input


        
    def processText(self,stc):
        result = stc
        
        flag = 0
        if result[-1]=='\n':
            result = result.replace('\n','')
        
        num = 0
        while 1:
            if result[-1-num]!=' ':
                break
            else:
                num = num+1
                
        
                
        if num==0:
            rere = result
        else:
            rere = result[:-num]
            
        
        r_pun = ''
        r_word = rere
        while True:
            if r_word[-1] in SV_LIST:
                r_pun = r_pun+r_word[-1]
                r_word = r_word[:-1]
            else:
                break
        
        
        num_space = 0
        for i in r_word:
            if i==' ':
                num_space = num_space+1
            else:
                break
            
        if num_space!=0:
            r_word = r_word[num_space:]

#         cc = ch.processText(line)
    
        plus = ''
        for s in range(num_space):
            plus = plus+' '
    
        if r_word[-1] =='?' or r_word[-1] =='.' or r_word[-1] =='!' or r_word[-1] =='\"':
            r_word = r_word
        else:
            r_word = r_word+'.'
            flag = 1
        try:
            res = self.to_high(r_word)
        except:
            res = r_word
            
        r_word = plus+r_word
        res = plus+res
        
        
        
#         if r_word[-1] =='?' or r_word[-1] =='.' or r_word[-1] =='!' or r_word[-1] =='\"':
#             r_word = r_word
#         else:
#             r_word = r_word+'.'
#             flag = 1
        
#         res = self.to_high(r_word)
        
        if flag ==1:
            res = res[:-1]
        
        #spacing = Spacing()
        #res = spacing(res)
        return res+r_pun[::-1]

In [9]:
ch = Changer()
ch.to_high('난 그걸 잘 몰라.')

'난 그걸 잘 몰라요.'

In [10]:
f0 = open('./우효/aihub.kr.shuf', 'r', encoding = 'utf-8')
f1 = open('./우효/aihub.mecab.shuf01', 'w',encoding = 'utf-8')



flag = 0

num = 0
number = 0
ch = Changer()

while True:
    flag = 0
    line = f0.readline()
    if not line: break
    if line[-1]=='\n':
        line = line.replace('\n','')
    
    num_space = 0
    for i in line:
        if i==' ':
            num_space = num_space+1
        else:
            break
            
    if num_space!=0:
        line = line[num_space:]
    
#     try:
    cc = ch.processText(line)
#     except:
#         cc = line
    
    plus = ''
    for s in range(num_space):
        plus = plus+' '
    
    line = plus+line
    cc = plus+cc
    f1.write(cc)
    f1.write('\n')
    num = num+1
    if cc!=line:
        number = number+1
    if num%1000==0:
        print(num)
#     else:
#         break
#    print(number)

f0.close()
f1.close()

FileNotFoundError: [Errno 2] No such file or directory: './우효/aihub.kr.shuf'

In [11]:
f0 = open('./우효/aihub.mecab.shuf01', 'r', encoding = 'utf-8')
f1 = open('./우효/aihub.mecab.shuf', 'w',encoding = 'utf-8')



flag = 0

num = 0
number = 0
ch = Changer()

while True:
    flag = 0
    line = f0.readline()
    if not line: break
    if line[-1]=='\n':
        line = line.replace('\n','')
    
    
#     try:
    if 'ㅂ니다' in line:
        cc = line.replace('ㅂ니다', '입니다')
    else:
        cc = line
#     except:
#         cc = line
    
    
    
    f1.write(cc)
    f1.write('\n')
    num = num+1
    if cc!=line:
        number = number+1
        print(cc)
        print(line)
    if num%1000==0:
        print(num)
#     else:
#         break
#    print(number)

f0.close()
f1.close()

FileNotFoundError: [Errno 2] No such file or directory: './우효/aihub.mecab.shuf01'

In [200]:
kr = open('./hgu_clean.mecab.shuf.shuf', "r" , encoding = "utf-8")

부호 = ['.', '?', '!', ')', '"', '…', '”', '．', ':', '~', ']', '┃', ',', '」']

요 = 0
다 = 0
까 = 0
나머지 = 0

result=kr.readline()

while(result != ''):
    result = result.replace('\n','')
    while result[-1] in 부호:
            result= result[:-1].strip()
    
    if '다요' in result:
        나머지+=1
        print(result)
        
    result=kr.readline()

kr.close()

하나 아니면 둘 다요 아니면 전혀 다른 걸 수도 있죠
매 분기마다요
맞아요 두 파일 캐비넷 모두 다요
두 번째는 빅데이터와 기계 학습의 시대라는 것입니다 기계 학습은 우리의 이해력에 혁명을 가져올 것을 약속합니다 소셜네트워크부터 역학까지 전부 다요
시동 걸 때마다요
전 갖가지 괴상한 실험도 많이 한답니다 음식에다요 이 경우엔 수학적 모델을 측정하려는 중입니다 누구든 내부 요리 시간이 얼마나 되는지 추정할 수 있게요
세상에 대한 그들의 영향력은 대단치 않았죠 해파리 개똥벌래 또는 딱다구리보다요
온갖 것에 관심을 기울이지요 움직이는 건 거의 다요 잠자리도 마찬가지입니다
이런 곡을 들을 때마다 이런 맙소사 같은 방에 있다니 믿어지지 않네요 이런 일이 일어나고 있다는게 믿기지 않습니다 이건 더 좋은 방식의 경험입니다 어떤 형식으로든 여러분들이 들어오던 방식보다요
하지만 날개 기저에 붙은 아주 작은 조종 근육이 있습니다 힘은 전혀 세지 않지만 대신 무척 빠르죠 그리고 날개가 접히는 정도를 바꿀 수 있게 해줍니다 그것도 날개짓을 할 때마다요 이것이 파리가 날개를 바꿔 공기역학적 힘을 변화시킴으로써 비행궤적을 바꾸는 방법입니다
이것이 소리를 낼 겁니다 목소리나 음악 소리를 들을 때마다요 이건 제가 버튼을 눌렀기 때문입니다
음, 먼저, 톨레도보다는, 세고비아를, 그리고 그라나다요
그래서 그녀는 왜 대학을 자퇴했다요
마스터님 돌을 움직이는 것 중 하나입니다요
저는 거기서 그 사람들과 포경캠프에서 같이 지내면서 전 과정을 다 사진으로 기록했는데요 뉴욕에서 뉴왁공항으로 가는 택시를 타는 것부터 시작해서 7 일반 이후 두 번째 고래를 도축하는 마지막까지 모두 다요
하지만 제가 보여 드릴것은 다른 방법으로 어려움을 없애고 정답에 도달 하는 것입니다 아마도 아서보다 빠를것입니다 첫번째로 상식과 논리에 반할 것입니다 모두다 손을 이렇게 해보세요 90도요 모두다요
아무렴 좋다 마다요
책 하나만요 아님 전부다요
매 30분마다요
판매 목표치 달성했어요” 거의 다요
앞은 바다요 뒤는 산
여러분

In [25]:
def prepro_ch02(input, lis_ef, tag_last, lis_w_last, lis_w_last_not, lis_ic):
    
    lis_res_word = []
    
    lis_input = input.split()
    lis_word = []
    lis_tag = []
    lis_last_word = []
    lis_ind = []
    
    lis_result = []
    
    for i in range(len(lis_input)):
        ele_w = []
        ele_t = []
        
        an = mec.pos(lis_input[i])
        for j in range(len(an)):
            ele_w.append(an[j][0])
            ele_t.append(an[j][1])
        elem_w = ' '.join(ele_w)
        elem_t = '/'.join(ele_t)
        
        jam_pre = Jamodealer(elem_w)
        lis_word.append(''.join(jam_pre.jamo))
        lis_tag.append(elem_t)
    
    for i in range(len(lis_tag)):
        
        if 'EF/SF' in lis_tag[i] or 'EF/SV' in lis_tag[i] or 'EF/SC' in lis_tag[i] or 'JX/SF' in lis_tag[i] or 'JX/SC' in lis_tag[i] or 'IC' in lis_tag[i] or 'IC' in lis_tag[i]:
            elemen_t = lis_tag[i].split('/')
            elemen_w = lis_word[i].split(' ')
            flag = 0
            
            
            for j in range(len(elemen_t)):

                flag_end = detect_h(elemen_w[j], lis_w_last_not, lis_w_last)

                
                if (flag_end == 1 and 'EF' in elemen_t[j]) or (flag_end == 1 and 'JX' in elemen_t[j]):
                    
                    for jam in lis_w_last_not:
                        
                        if len(elemen_w[j])>=len(jam):
                            
                            if elemen_w[j][-len(jam):]==jam:
                                
                                
                                lis_ind.append(i)
                                lis_last_word.append(jam)
                                elemen_w[j] = elemen_w[j][:-len(jam)]
                                
                                break
                                
                elif 'IC' in elemen_t[j] or 'IC' in elemen_t[j]:
                    
                    for ii in lis_ic:
                        
                        if len(elemen_w[j])>=len(ii):
                            
                            if elemen_w[j][-len(ii):]==ii:
                                
                                
                                lis_ind.append(i)
                                lis_last_word.append(ii)
                                elemen_w[j] = elemen_w[j][:-len(ii)]
                                
                                break
                                
            res_w = ''.join(elemen_w)
            lis_result.append(res_w)
            
        elif 'NP' in lis_tag[i] and ('ㅈㅓ' in lis_word[i] or 'ㅈㅔ' in lis_word[i]):
            
            elemen_t = lis_tag[i].split('/')
            elemen_w = lis_word[i].split(' ')
            flag = 0
            
            for ind in range(len(elemen_t)):
                
                if flag==0:
                    
                    if elemen_t[ind]=='NP':
                        
                        if elemen_w[ind] == 'ㅈㅓ':
                            
                            elemen_w[ind] = 'ㄴㅏ'
                            flag = 1
                        
                        elif elemen_w[ind] == 'ㅈㅔ':
                            
                            elemen_w[ind] = 'ㄴㅐ'
                            flag = 1
            
            resres = ''.join(elemen_w)
            
            lis_result.append(resres)
            
                                
        else:
            
            rere = lis_word[i].split(' ')
            
            resres = ''.join(rere)
            
            lis_result.append(resres)
            
            
    return lis_result, lis_tag, lis_ind, lis_last_word
     

In [32]:
print(prepro_ch02('저는 당신을 사랑합니다.', lis_beta_ef, lis_tag_last, lis_end_2low, lis_end, lis_ic))

(['ㄴㅏㄴㅡㄴ', 'ㄷㅏㅇㅅㅣㄴㅇㅡㄹ', 'ㅅㅏㄹㅏㅇㅎㅏ.'], ['NP/JX', 'NP/JKO', 'NNG/XSV+EF/SF'], [2], ['ㅂㄴㅣㄷㅏ'])
